In [3]:
import pandas as pd
import numpy as np

# Configuration for accelerators and models
accelerators = ["nvdla", "shidiannao", "eyeriss"]
models = [
    "test_layer_VGG16_conv1",
    "test_layer_BERT_qk",
    "test_layer_BERT_ffn",
    "test_model_YOLOv8",
    "test_model_MobileNetv3",
    "test_model_MobileViT",
    "test_model_VGG16",
    "test_model_InceptionV3"
]

dataflow_mapping = {
    "nvdla": "ykp_os",
    "shidiannao": "kcp_ws",
    "eyeriss": "rs"
}

freq_MHz = 350  # Frequency in MHz
freq_Hz = freq_MHz * 1e6

# Results storage
results = []

for accelerator in accelerators:
    
    for model in models:
        # Dynamically construct the file path
        file_path = f'out/{accelerator}/{accelerator}_{model}_{dataflow_mapping[accelerator]}.csv'
        
        try:
            # Read CSV file
            df = pd.read_csv(file_path)

            # Extract column indices dynamically
            runtime_col = df.columns.get_loc(' Runtime (Cycles)')
            input_l1_read_col = df.columns.get_loc(' input l1 read')
            input_l1_write_col = df.columns.get_loc(' input l1 write')
            filter_l1_read_col = df.columns.get_loc('filter l1 read')
            filter_l1_write_col = df.columns.get_loc(' filter l1 write')
            output_l1_read_col = df.columns.get_loc('output l1 read')
            output_l1_write_col = df.columns.get_loc(' output l1 write')
            input_l2_read_col = df.columns.get_loc(' input l2 read')
            input_l2_write_col = df.columns.get_loc(' input l2 write')
            filter_l2_read_col = df.columns.get_loc(' filter l2 read')
            filter_l2_write_col = df.columns.get_loc(' filter l2 write')
            output_l2_read_col = df.columns.get_loc(' output l2 read')
            output_l2_write_col = df.columns.get_loc(' output l2 write')
            offchip_bw_col = df.columns.get_loc(' Offchip BW Req (Elements/cycle)')
            reuse_input_col = df.columns.get_loc(' input reuse factor')
            reuse_filter_col = df.columns.get_loc(' filter reuse factor')
            reuse_output_col = df.columns.get_loc(' output reuse factor')
            num_macs_col = df.columns.get_loc(' Num MACs')

            # Convert DataFrame to NumPy array for vectorized operations
            data = df.to_numpy()

            # Perform vectorized calculations
            cycles = np.sum(data[:, runtime_col])
            num_macs = np.sum(data[:, num_macs_col])
            l1_access = np.sum(data[:, input_l1_read_col] + data[:, input_l1_write_col] +
                            data[:, filter_l1_read_col] + data[:, filter_l1_write_col] +
                            data[:, output_l1_read_col] + data[:, output_l1_write_col])
            l2_access = np.sum(data[:, input_l2_read_col] + data[:, input_l2_write_col] + 
                            data[:, filter_l2_read_col] + data[:, filter_l2_write_col] +
                            data[:, output_l2_read_col] + data[:, output_l2_write_col])
            dram_access = np.sum(data[:, offchip_bw_col] * data[:, runtime_col])
            
            energy_nJ = num_macs * 0.48 + l1_access * 0.15 + l2_access * 3.69 + dram_access * 31.2 * 1e-3 # pJ to nJ
            delay_s = cycles / (freq_MHz * 1e6)
            edp_nJ = energy_nJ * delay_s
            averaged_reuse_factor = np.mean((data[:, reuse_input_col] + data[:, reuse_filter_col] + data[:, reuse_output_col]) / 3)
            weighted_memory_access = l1_access + 5 * l2_access + 10 * dram_access
            leakage_mW = 100 
            power_proxy_value = (energy_nJ) / cycles * freq_MHz + leakage_mW

            # Append results for this file
            results.append({
                'Accelerator': accelerator,
                'Model': model,
                'Cycles': cycles,
                'Runtime (s)': f"{delay_s:.4f}",
                'EDP': f"{edp_nJ:.4f}",
                'Memory Access': weighted_memory_access,
                'Reuse Factor': f"{averaged_reuse_factor:.4f}",
                'Power Proxy': f"{power_proxy_value:.4f}",
            })

        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('out/summary_results_existing_accelerators_.csv', index=False)

# Display summary results
print(results_df)


   Accelerator                   Model     Cycles Runtime (s)  \
0        nvdla  test_layer_VGG16_conv1     612721      0.0018   
1        nvdla      test_layer_BERT_qk      13313      0.0000   
2        nvdla     test_layer_BERT_ffn     294913      0.0008   
3        nvdla       test_model_YOLOv8    4571206      0.0131   
4        nvdla  test_model_MobileNetv3    1132209      0.0032   
5        nvdla    test_model_MobileViT     323590      0.0009   
6        nvdla        test_model_VGG16   48072056      0.1373   
7        nvdla  test_model_InceptionV3    2843144      0.0081   
8   shidiannao  test_layer_VGG16_conv1   15770881      0.0451   
9   shidiannao      test_layer_BERT_qk      24576      0.0001   
10  shidiannao     test_layer_BERT_ffn      55296      0.0002   
11  shidiannao       test_model_YOLOv8   16137681      0.0461   
12  shidiannao  test_model_MobileNetv3    5648773      0.0161   
13  shidiannao    test_model_MobileViT     602112      0.0017   
14  shidiannao        tes

In [2]:
import pandas as pd
import numpy as np

# Configuration for accelerators and models
accelerators = ["nvdla", "shidiannao", "eyeriss"]
models = [
    "test_layer_VGG16_conv1",
    "test_layer_BERT_qk",
    "test_layer_BERT_ffn",
    "test_model_YOLOv8",
    "test_model_MobileNetv3",
    "test_model_MobileViT",
    "test_model_VGG16",
    "test_model_InceptionV3"
]

dataflow_mapping = {
    "nvdla": "ykp_os",
    "shidiannao": "kcp_ws",
    "eyeriss": "rs"
}

freq_MHz = 350  # Frequency in MHz
freq_Hz = freq_MHz * 1e6

# Results storage
results = []

for accelerator in accelerators:
    for model in models:
        # Dynamically construct the file path
        file_path = f'out/{accelerator}/{accelerator}_{model}_{dataflow_mapping[accelerator]}.csv'
        
        try:
            # Read CSV file
            df = pd.read_csv(file_path)

            # Extract column indices dynamically
            runtime_col = df.columns.get_loc(' Runtime (Cycles)')
            energy_col = df.columns.get_loc(' Activity count-based Energy (nJ)')
            input_l1_read_col = df.columns.get_loc(' input l1 read')
            input_l1_write_col = df.columns.get_loc(' input l1 write')
            filter_l1_read_col = df.columns.get_loc('filter l1 read')
            filter_l1_write_col = df.columns.get_loc(' filter l1 write')
            output_l1_read_col = df.columns.get_loc('output l1 read')
            output_l1_write_col = df.columns.get_loc(' output l1 write')
            input_l2_read_col = df.columns.get_loc(' input l2 read')
            input_l2_write_col = df.columns.get_loc(' input l2 write')
            filter_l2_read_col = df.columns.get_loc(' filter l2 read')
            filter_l2_write_col = df.columns.get_loc(' filter l2 write')
            output_l2_read_col = df.columns.get_loc(' output l2 read')
            output_l2_write_col = df.columns.get_loc(' output l2 write')
            offchip_bw_col = df.columns.get_loc(' Offchip BW Req (Elements/cycle)')
            reuse_input_col = df.columns.get_loc(' input reuse factor')
            reuse_filter_col = df.columns.get_loc(' filter reuse factor')
            reuse_output_col = df.columns.get_loc(' output reuse factor')
            num_macs_col = df.columns.get_loc(' Num MACs')

            # Convert DataFrame to NumPy array for vectorized operations
            data = df.to_numpy()

            # Perform vectorized calculations
            cycles = np.sum(data[:, runtime_col])
            num_macs = np.sum(data[:, num_macs_col])
            l1_access = np.sum(data[:, input_l1_read_col] + data[:, input_l1_write_col] +
                            data[:, filter_l1_read_col] + data[:, filter_l1_write_col] +
                            data[:, output_l1_read_col] + data[:, output_l1_write_col])
            l2_access = np.sum(data[:, input_l2_read_col] + data[:, input_l2_write_col] + 
                            data[:, filter_l2_read_col] + data[:, filter_l2_write_col] +
                            data[:, output_l2_read_col] + data[:, output_l2_write_col])
            dram_access = np.sum(data[:, offchip_bw_col] * data[:, runtime_col])
            
            # Calculate energy using the provided formula
            energy_nJ = num_macs * 0.48 + l1_access * 0.15 + l2_access * 3.69 + dram_access * 31.2 * 1e-3
            
            # Delay and EDP calculations
            delay_s = cycles / freq_Hz
            edp_nJ = energy_nJ * delay_s
            
            # Reuse factor (average across all layers)
            reuse_factors = (data[:, reuse_input_col] + data[:, reuse_filter_col] + data[:, reuse_output_col]) / 3
            
            # Average reuse factor
            if len(data) > 1:
                averaged_reuse_factor = np.mean(reuse_factors)
            else:
                averaged_reuse_factor = np.sum(reuse_factors)  # For single row, sum = original value
            
            # Weighted memory access (sum across all layers)
            weighted_memory_access = l1_access + 5 * l2_access + 10 * dram_access
            
            # Power proxy value (average across all layers)
            leakage_mW = 100
            power_proxy_value = (energy_nJ / cycles) * freq_MHz + leakage_mW

            # Append results for this file
            results.append({
                'Accelerator': accelerator,
                'Model': model,
                'Cycles': cycles,  # Sum
                'Runtime (s)': f"{delay_s:.4f}",  # Sum
                'EDP': f"{edp_nJ:.4f}",  # Sum or Mean
                'Memory Access': weighted_memory_access,  # Sum
                'Reuse Factor': f"{averaged_reuse_factor:.4f}",  # Average
                'Power Proxy': f"{power_proxy_value:.4f}",
            })

        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('out/summary_results_existing_accelerators_.csv', index=False)

# Display summary results
print(results_df)


   Accelerator                   Model     Cycles Runtime (s)  \
0        nvdla  test_layer_VGG16_conv1     612721      0.0018   
1        nvdla      test_layer_BERT_qk      13313      0.0000   
2        nvdla     test_layer_BERT_ffn     294913      0.0008   
3        nvdla       test_model_YOLOv8    4571206      0.0131   
4        nvdla  test_model_MobileNetv3    1132209      0.0032   
5        nvdla    test_model_MobileViT     323590      0.0009   
6        nvdla        test_model_VGG16   48072056      0.1373   
7        nvdla  test_model_InceptionV3    2843144      0.0081   
8   shidiannao  test_layer_VGG16_conv1   15770881      0.0451   
9   shidiannao      test_layer_BERT_qk      24576      0.0001   
10  shidiannao     test_layer_BERT_ffn      55296      0.0002   
11  shidiannao       test_model_YOLOv8   16137681      0.0461   
12  shidiannao  test_model_MobileNetv3    5648773      0.0161   
13  shidiannao    test_model_MobileViT     602112      0.0017   
14  shidiannao        tes

In [12]:
import os
import pandas as pd

def process_metric_with_generations(dir_path, model, metric, num_generations=100):
    # Initialize an empty list to store metrics for each file
    metric_data = []
    
    # Loop through all files in the directory
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.csv'):  # Check if it's a CSV file
            file_path = os.path.join(dir_path, file_name)
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Check if the metric exists in the file
            if metric in df.columns:
                # Ensure the number of generations matches the expected count
                metric_values = df[metric].tolist()[:num_generations]
                if len(metric_values) < num_generations:
                    # Pad with NaN if the file has fewer generations
                    metric_values.extend([float('nan')] * (num_generations - len(metric_values)))
                metric_data.append(metric_values)
    
    # Prepare the output DataFrame without generation indices
    output_df = pd.DataFrame(metric_data)
    
    # Define the output path
    output_file_name = f"out/generations_{model}_{metric.replace(' ', '_').lower()}.csv"
    os.makedirs('out', exist_ok=True)  # Ensure the output directory exists
    output_df.to_csv(output_file_name, index=False)
    print(f"Saved {metric} to {output_file_name}")

# Example usage
dir_path = 'out/magneto_power_constraint/VGG16_conv1/'
model = 'VGG16_conv1'
metrics = ['Best Fitness Score', 'Best EDP', 'Best Power Proxy', 'Best Reuse Factor']	
num_generations = 100  # Adjust this if the number of generations is different

for metric in metrics:
    process_metric_with_generations(dir_path, model, metric)


Saved Best Fitness Score to out/generations_VGG16_conv1_best_fitness_score.csv
Saved Best EDP to out/generations_VGG16_conv1_best_edp.csv
Saved Best Power Proxy to out/generations_VGG16_conv1_best_power_proxy.csv
Saved Best Reuse Factor to out/generations_VGG16_conv1_best_reuse_factor.csv
